<a href="https://colab.research.google.com/github/StratagemGIS/notebooks/blob/main/projects/38_use_geoprocessing_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> Reference </h1>

https://pro.arcgis.com/en/pro-app/latest/get-started/use-geoprocessing-tools.htm

<img src="https://pro.arcgis.com/en/pro-app/latest/get-started/GUID-9897870A-9126-48A2-8961-BB89E8BBBFC8-web.png" width=300 height=200>

Nassella tussock is an invasive weed accidentally brought to New Zealand in the late 1800s. It is now well established in the Marlborough and Canterbury regions on the South Island. It spreads quickly, crowds out other grasses, and is hard to eradicate. It is unpalatable to livestock, which makes it a threat to pastoral farming. Nassella tussock seeds are spread mainly by wind, but also by animals, people, and vehicles.

In this tutorial, you'll look for Nassella tussock growing near commercial campgrounds (campgrounds for motorhomes). Campers who come in contact with the weed during recreational activities may get seeds in their clothing or gear and take these seeds with them when they leave. If your analysis shows that Nassella tussock grows near commercial campgrounds, agriculture officials can provide information to help campers identify and avoid it.

The analysis has three parts. First, you'll create buffers, or proximity zones, around the commercial campgrounds. Next, you'll see which of these buffers contain Nassella tussock. Finally, you'll calculate the amount of Nassella tussock within the buffers.

Vaasudevan Srinivasan 🧑🏻‍💻  
StratagemGIS Solutions

In [ ]:
%%capture

!pip install py7zr

import fiona
import geopandas as gpd
import pooch
import py7zr

In [ ]:
ppkx_file = pooch.retrieve(
    'https://raw.githubusercontent.com/StratagemGIS/datasets/main/esri/ppkx/Use_geoprocessing_tools.ppkx',
    known_hash='b293ae30e7cde754eb5edcc9dc1aebb49e2b0bfe18f0f5d9605dc2a10514f4c7'
)

with py7zr.SevenZipFile(ppkx_file) as archive:
    archive.extractall('Use_geoprocessing_tools')
    gdb_file = 'Use_geoprocessing_tools/p30/use_geoprocessing_tools.gdb'
    print(fiona.listlayers(gdb_file))

['NasellaTussock', 'Places', 'Roads', 'Commercial_Campgrounds', 'Regions']


In [ ]:
nasella_tussock = gpd.read_file(gdb_file, layer=0)
campgrounds = gpd.read_file(gdb_file, layer=3)

result = (
    campgrounds.assign(geometry=campgrounds.buffer(1.5 * 1000))
    .sjoin(
        nasella_tussock.assign(nasella_geom=nasella_tussock.geometry),
        predicate='intersects',
    )
    .rename(columns={'geometry': 'campground_buff_geom'})
    .assign(summarized_area=lambda df_:
        df_.campground_buff_geom.intersection(df_.nasella_geom).area.div(10_000)
    )
    .groupby(['Name', 'Address'], as_index=False)
    .agg({'summarized_area': 'sum'})
)

result

,Name,Address,summarized_area
0,A1 Ward Motel & Campground,"7305 SH 1, Ward",72.232399
1,Awatere Settlers Motor Camp,"Seymour Street, Seddon",83.208572
2,Blenheim TOP 10 Holiday Park,"78 Grove Road, Blenheim",0.249217
3,Chartridge Park,"3389 SH 6, Havelock",0.678513
